In [ ]:
import os
import pickle
import re
import json
from tqdm import tqdm
import math
from Levenshtein import distance
from utils import rm_idx, match_idx
from metrics import Metrics


def load_pickle(filename):
    if filename is None or not os.path.exists(filename):
        return None
    with open(filename, "rb") as f:
        return pickle.load(f)


def save_pickle(data, filename):
    if filename is None:
        return
    with open(filename, "wb") as f:
        pickle.dump(data, f, protocol=pickle.HIGHEST_PROTOCOL)


In [ ]:
# evaluating SFTTestSeqRec Top10 gpt-3.5-turbo-1106_output
res_file = [
    '../data/dataset/sub_movie/GPT_datum_info_test_SFTTestSeqRec_LCT_Top10.pickle_Result0-999.jsonl',
    '../data/dataset/sub_movie/GPT_datum_info_test_SFTTestSeqRec_LCT_Top10.pickle_Result1000-1499.jsonl',
    '../data/dataset/sub_movie/GPT_datum_info_test_SFTTestSeqRec_LCT_Top10.pickle_Result1500-1999.jsonl',
    '../data/dataset/sub_movie/GPT_datum_info_test_SFTTestSeqRec_LCT_Top10.pickle_Result2000-3999.jsonl',
    '../data/dataset/sub_movie/GPT_datum_info_test_SFTTestSeqRec_LCT_Top10.pickle_Result4000-4999.jsonl',
    '../data/dataset/sub_movie/GPT_datum_info_test_SFTTestSeqRec_LCT_Top10.pickle_gpt-3.5-turbo-1106_Result5000-7199.jsonl',
]
meta = load_pickle('../data/dataset/steam/meta1.pickle')
title = [meta[_]['title'] for _ in meta]
title = [re.sub(r' +', ' ', _) for _ in title]
title = [re.sub(r': +', ':', _) for _ in title]
title = [re.sub(r', +', ',', _) for _ in title]
gpt_seq_rec_data_top10 = []
for _ in res_file:
    old_len = len(gpt_seq_rec_data_top10)
    for idx, __ in enumerate(open(_, mode='r', encoding="utf-8").readlines()):
        try:
            gpt_seq_rec_data_top10.append(json.loads(__))
        except Exception as e:
            print('error: ', e, '\nidx: ', idx, 'of', _)
    print(_, f' | count: {len(gpt_seq_rec_data_top10)-old_len}')

k = 10
recall, mrr, ndcg, repeat, not_exist, in_history, correct_count = 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0
count = len(gpt_seq_rec_data_top10)
for _ in tqdm(gpt_seq_rec_data_top10):
    if _['gpt-3.5-turbo-1106_output'][0] == _['gpt-3.5-turbo-1106_output'][-1] == '"' or _['gpt-3.5-turbo-1106_output'][0] == _['gpt-3.5-turbo-1106_output'][-1] == "'":
        _['gpt-3.5-turbo-1106_output'] = _['gpt-3.5-turbo-1106_output'][1:-1]
    history = re.sub(r' +', ' ', _['input_field_data']['history'])
    history = re.sub(r': +', ':', history)
    history = re.sub(r', +', ',', history)
    if ' 鈫?' in history:
        history = [__[1:-1] for __ in history.split(' 鈫?')]
    else:
        history = [__[1:-1] for __ in history.split(' → ')]
    assert len(history) > 1

    label = rm_idx(_['output_text'])
    label = re.sub(r' +', ' ', label)
    label = re.sub(r': +', ':', label)
    label = re.sub(r', +', ',', label)

    ts = _['gpt-3.5-turbo-1106_output'].split('\n')
    if len(ts) == 1:
        ts = _['gpt-3.5-turbo-1106_output'].split("', '")
    ts = [rm_idx(__).strip().split('\n')[0].strip() for __ in ts]
    ts = [__[2:] if __.startswith('- ') else __ for __ in ts]
    ts = [re.sub(r' +', ' ', __) for __ in ts]
    ts = [re.sub(r': +', ':', __) for __ in ts]
    ts = [re.sub(r', +', ',', __) for __ in ts]
    ts = [__[1:-1] if __[0] == __[-1] == "'" or __[0] == __[-1] == "\"" else __ for __ in ts if __ != '']
    ts = [__.strip() for __ in ts]
    ts = ts[:k]
    # for idx, __ in enumerate(ts):
    #     if __ in title:
    #         continue
    #     for ___ in title:
    #         if distance(__, ___) <= 3:
    #             ts[idx] = ___
    #             break
    recall += 1 if label in ts else 0
    mrr += 1/(ts.index(label)+1) if label in ts else 0
    ndcg += 1/math.log2(ts.index(label)+2) if label in ts else 0
    repeat += sum([1 for idx, __ in enumerate(ts) if __ in ts[:idx]])
    not_exist += sum([1 for __ in ts if __ not in title])
    in_history += sum([1 for __ in ts if __ in history])
    # if sum([1 for __ in ts if __ not in title]) > 0:
    #     print(_['gpt-3.5-turbo-1106_output'])
    #     print([__ for __ in ts if __ not in title])
    #     print('')
    correct_count += 1 if len(ts) == _['input_field_data']['item_count'] else 0
print(f'''top_k: {k}
count: {len(gpt_seq_rec_data_top10)}
recall: {recall/len(gpt_seq_rec_data_top10):.5f}
mrr: {mrr/len(gpt_seq_rec_data_top10):.5f}
ndcg: {ndcg/len(gpt_seq_rec_data_top10):.5f}
repeat: {repeat/len(gpt_seq_rec_data_top10):.5f}
not_exist: {not_exist/len(gpt_seq_rec_data_top10):.5f}
in_history: {in_history/len(gpt_seq_rec_data_top10):.5f}
correct_count: {correct_count/len(gpt_seq_rec_data_top10):.5f}''')

In [ ]:
# evaluating SFTTestSeqRanking Top5 gpt-3.5-turbo-1106_output
res_file = [
    '../data/dataset/sub_movie/GPT_datum_info_test_SFTTestSeqRanking_LCT_Top5.pickle_Result0-9.jsonl',
    '../data/dataset/sub_movie/GPT_datum_info_test_SFTTestSeqRanking_LCT_Top5.pickle_Result10-2999.jsonl',
    '../data/dataset/sub_movie/GPT_datum_info_test_SFTTestSeqRanking_LCT_Top5.pickle_Result3000-4999.jsonl',
    '../data/dataset/sub_movie/GPT_datum_info_test_SFTTestSeqRanking_LCT_Top5.pickle_gpt-3.5-turbo-1106_Result5000-6599.jsonl',
]
gpt_seq_ranking_data_top5 = []
for _ in res_file:
    old_len = len(gpt_seq_ranking_data_top5)
    for idx, __ in enumerate(open(_, mode='r', encoding="utf-8").readlines()):
        try:
            gpt_seq_ranking_data_top5.append(json.loads(__))
        except Exception as e:
            print('error: ', e, '\nidx: ', idx, 'of', _)
    print(_, f' | count: {len(gpt_seq_ranking_data_top5)-old_len}')

k = 5
recall, mrr, ndcg, repeat, correct_count, out_of_candidate = 0.0, 0.0, 0.0, 0.0, 0.0, 0.0
count = len(gpt_seq_ranking_data_top5)
for _ in tqdm(gpt_seq_ranking_data_top5):
    if _['gpt-3.5-turbo-1106_output'][0] == _['gpt-3.5-turbo-1106_output'][-1] == '"' or _['gpt-3.5-turbo-1106_output'][0] == _['gpt-3.5-turbo-1106_output'][-1] == "'":
        _['gpt-3.5-turbo-1106_output'] = _['gpt-3.5-turbo-1106_output'][1:-1]
    candidates = re.sub(r' +', ' ', _['input_field_data']['candidate_titles'])
    candidates = re.sub(r': +', ':', candidates)
    candidates = re.sub(r', +', ',', candidates)
    candidates = [__[1:-1] for __ in candidates.split(',')]

    label = rm_idx(_['output_text'])
    label = re.sub(r' +', ' ', label)
    label = re.sub(r': +', ':', label)
    label = re.sub(r', +', ',', label)

    ts = _['gpt-3.5-turbo-1106_output'].split('\n')
    if len(ts) == 1:
        ts = _['gpt-3.5-turbo-1106_output'].split("', '")
    ts = [rm_idx(__).strip().split('\n')[0].strip() for __ in ts]
    ts = [__[2:] if __.startswith('- ') else __ for __ in ts]
    ts = [re.sub(r' +', ' ', __) for __ in ts]
    ts = [re.sub(r': +', ':', __) for __ in ts]
    ts = [re.sub(r', +', ',', __) for __ in ts]
    ts = [__[1:-1] if __[0] == __[-1] == "'" or __[0] == __[-1] == "\"" else __ for __ in ts if __ != '']
    ts = [__.strip() for __ in ts]
    ts = ts[:k]

    for idx, __ in enumerate(ts):
        if __ in candidates:
            continue
        for ___ in candidates:
            if distance(__, ___) <= 3:
                ts[idx] = ___
    recall += 1 if label in ts else 0
    mrr += 1/(ts.index(label)+1) if label in ts else 0
    ndcg += 1/math.log2(ts.index(label)+2) if label in ts else 0
    repeat += sum([1 for idx, __ in enumerate(ts) if __ in ts[:idx]])
    out_of_candidate += sum([1 for __ in ts if __ not in candidates])
    # if sum([1 for __ in ts if __ not in candidates]) > 0:
    #     print(_['gpt-3.5-turbo-1106_output'])
    #     print([__ for __ in ts if __ not in candidates], candidates)
    #     print('')
    correct_count += 1 if len(ts) == _['input_field_data']['item_count'] else 0
print(f'''top_k: {k}
count: {count}
recall: {recall/count:.5f}
mrr: {mrr/count:.5f}
ndcg: {ndcg/count:.5f}
repeat: {repeat/count:.5f}
out_of_candidate: {out_of_candidate/count:.5f}
correct_count: {correct_count/count:.5f}''')

In [ ]:
# evaluating SFTTestSeqRec Top10 Llama-2-7b-hf-chat_output
res_file = [
    '../data/dataset/sub_movie/GPT_datum_info_test_SFTTestSeqRec_LCT_Top10.pickle_Llama-2-7b-hf-chat_Result0-13217.jsonl',
]
meta = load_pickle('../data/dataset/steam/meta1.pickle')
title = [meta[_]['title'] for _ in meta]
title = [re.sub(r' +', ' ', _) for _ in title]
title = [re.sub(r': +', ':', _) for _ in title]
title = [re.sub(r', +', ',', _) for _ in title]
gpt_seq_rec_data_top10 = []
for _ in res_file:
    old_len = len(gpt_seq_rec_data_top10)
    for idx, __ in enumerate(open(_, mode='r', encoding="utf-8").readlines()[:]):
        try:
            gpt_seq_rec_data_top10.append(json.loads(__))
        except Exception as e:
            print('error: ', e, '\nidx: ', idx, 'of', _)
    print(_, f' | count: {len(gpt_seq_rec_data_top10)-old_len}')

print(gpt_seq_rec_data_top10[0])
k = 10
recall, mrr, ndcg, repeat, not_exist, in_history, target_in_history, correct_count = 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0
count = len(gpt_seq_rec_data_top10)
for _ in tqdm(gpt_seq_rec_data_top10):
    if _['Llama-2-7b-hf-chat_output'][0] == _['Llama-2-7b-hf-chat_output'][-1] == '"' or _['Llama-2-7b-hf-chat_output'][0] == _['Llama-2-7b-hf-chat_output'][-1] == "'":
        _['Llama-2-7b-hf-chat_output'] = _['Llama-2-7b-hf-chat_output'][1:-1]
    history = re.sub(r' +', ' ', _['input_field_data']['history'])
    history = re.sub(r': +', ':', history)
    history = re.sub(r', +', ',', history)
    if ' â\x86\x92 ' in history:
        history = [__[1:-1] for __ in history.split(' â\x86\x92 ')]
    else:
        history = [__[1:-1] for __ in history.split(' → ')]
    assert len(history) > 1

    label = rm_idx(_['output_text'])
    label = re.sub(r' +', ' ', label)
    label = re.sub(r': +', ':', label)
    label = re.sub(r', +', ',', label)

    ts = _['Llama-2-7b-hf-chat_output'].split('\n')
    ts = [rm_idx(__).strip().split('\n')[0].strip() for __ in ts if match_idx(__)]
    ts = [re.sub(r' +', ' ', __) for __ in ts]
    ts = [re.sub(r': +', ':', __) for __ in ts]
    ts = [re.sub(r', +', ',', __) for __ in ts]
    ts = [__[1:-1] if __[0] == __[-1] == "'" or __[0] == __[-1] == "\"" else __ for __ in ts if __ != '']
    ts = [__.strip() for __ in ts]
    ts = ts[:k]

    for idx, __ in enumerate(ts):
        if __ in title:
            continue
        for ___ in title:
            if distance(__, ___) <= 3:
                ts[idx] = ___
                break
    recall += 1 if label in ts else 0
    mrr += 1/(ts.index(label)+1) if label in ts else 0
    ndcg += 1/math.log2(ts.index(label)+2) if label in ts else 0
    repeat += sum([1 for idx, __ in enumerate(ts) if __ in ts[:idx]])
    not_exist += sum([1 for __ in ts if __ not in title])
    in_history += sum([1 for __ in ts if __ in history])
    target_in_history += 1 if label in history else 0
    correct_count += 1 if len(ts) == _['input_field_data']['item_count'] else 0
print(f'''top_k: {k}
count: {len(gpt_seq_rec_data_top10)}
recall: {recall/len(gpt_seq_rec_data_top10):.5f}
mrr: {mrr/len(gpt_seq_rec_data_top10):.5f}
ndcg: {ndcg/len(gpt_seq_rec_data_top10):.5f}
repeat: {repeat/len(gpt_seq_rec_data_top10):.5f}
not_exist: {not_exist/len(gpt_seq_rec_data_top10):.5f}
in_history: {in_history/len(gpt_seq_rec_data_top10):.5f}
target_in_history: {target_in_history/len(gpt_seq_rec_data_top10):.5f}
correct_count: {correct_count/len(gpt_seq_rec_data_top10):.5f}''')

In [ ]:
# evaluating SFTTestSeqRec Top10 Llama-2-7b-hf-chat_output
res_file = [
    '../data/dataset/sub_movie/GPT_datum_info_test_SFTTestSeqRanking_LCT_Top5.pickle_Llama-2-7b-hf-chat_Result0-13217.jsonl',
]
gpt_seq_rec_data_top10 = []
for _ in res_file:
    old_len = len(gpt_seq_rec_data_top10)
    for idx, __ in enumerate(open(_, mode='r', encoding="utf-8").readlines()[:]):
        try:
            gpt_seq_rec_data_top10.append(json.loads(__))
        except Exception as e:
            print('error: ', e, '\nidx: ', idx, 'of', _)
    print(_, f' | count: {len(gpt_seq_rec_data_top10)-old_len}')

k = 5
recall, mrr, ndcg, repeat, correct_count, out_of_candidate = 0.0, 0.0, 0.0, 0.0, 0.0, 0.0
count = len(gpt_seq_rec_data_top10)
for _ in tqdm(gpt_seq_rec_data_top10):
    if _['Llama-2-7b-hf-chat_output'][0] == _['Llama-2-7b-hf-chat_output'][-1] == '"' or _['Llama-2-7b-hf-chat_output'][0] == _['Llama-2-7b-hf-chat_output'][-1] == "'":
        _['Llama-2-7b-hf-chat_output'] = _['Llama-2-7b-hf-chat_output'][1:-1]
    candidates = re.sub(r' +', ' ', _['input_field_data']['candidate_titles'])
    candidates = re.sub(r': +', ':', candidates)
    candidates = re.sub(r', +', ',', candidates)
    candidates = [__[1:-1] for __ in candidates.split(',')]

    label = rm_idx(_['output_text'])
    label = re.sub(r' +', ' ', label)
    label = re.sub(r': +', ':', label)
    label = re.sub(r', +', ',', label)

    ts = _['Llama-2-7b-hf-chat_output'].split('\n')
    ts = [rm_idx(__).strip().split('\n')[0].strip() for __ in ts if match_idx(__)]
    ts = [re.sub(r' +', ' ', __) for __ in ts]
    ts = [re.sub(r': +', ':', __) for __ in ts]
    ts = [re.sub(r', +', ',', __) for __ in ts]
    ts = [__[1:-1] if __[0] == __[-1] == "'" or __[0] == __[-1] == "\"" else __ for __ in ts if __ != '']
    ts = [__.strip() for __ in ts]
    ts = ts[:k]

    for idx, __ in enumerate(ts):
        if __ in candidates:
            continue
        for ___ in candidates:
            if distance(__, ___) <= 3:
                ts[idx] = ___
    recall += 1 if label in ts else 0
    mrr += 1/(ts.index(label)+1) if label in ts else 0
    ndcg += 1/math.log2(ts.index(label)+2) if label in ts else 0
    repeat += sum([1 for idx, __ in enumerate(ts) if __ in ts[:idx]])
    out_of_candidate += sum([1 for __ in ts if __ not in candidates])
    # if sum([1 for __ in ts if __ not in candidates]) > 0:
    #     print(_['Llama-2-7b-hf-chat_output'])
    #     print([__ for __ in ts if __ not in candidates], candidates)
    #     print('')
    correct_count += 1 if len(ts) == _['input_field_data']['item_count'] else 0
print(f'''top_k: {k}
count: {len(gpt_seq_rec_data_top10)}
recall: {recall/len(gpt_seq_rec_data_top10):.5f}
mrr: {mrr/len(gpt_seq_rec_data_top10):.5f}
ndcg: {ndcg/len(gpt_seq_rec_data_top10):.5f}
repeat: {repeat/len(gpt_seq_rec_data_top10):.5f}
out_of_candidate: {out_of_candidate/len(gpt_seq_rec_data_top10):.5f}
correct_count: {correct_count/len(gpt_seq_rec_data_top10):.5f}''')

In [ ]:
from concurrent.futures import ProcessPoolExecutor

# dataset, item_index = 'sub_movie', 'title64_t'
dataset, item_index = 'steam', 'title'
task, k = 'SFTTestSeqRec', 10
# task, k = 'SFT+TestPersonalControlRec', 10
# task, k = 'SFT-TestPersonalControlRec', 10
# task, k = 'SFTTestPersonalCategoryRate_30', 10
# task, k = 'SFTTestPersonalCategoryRate_50', 10
# task, k = 'SFTTestPersonalCategoryRate_70', 10

model = f'snap/Llama-2-7b-hf-chat/'
category2item = load_pickle(f'../data/dataset/{dataset}/category1.pickle')
meta = load_pickle(f'../data/dataset/{dataset}/meta1.pickle')
res_file = [
    f"/home/lws/projects/InstructControllableRec_RLHF/{model}{dataset}/{task}_Top10_Result_Sample.pickle"
]

title2item = {}
for _ in meta:
    if title2item.get(meta[_][item_index]) is None:
        title2item[meta[_][item_index]] = []
    title2item[meta[_][item_index]].append(_)

def vague_mapping(d):
    if d[f'{model}_output'] == "":
        d[f'{task}_output_title_list'] = []
        return d

    if d[f'{model}_output'][0] == d[f'{model}_output'][-1] == '"' or d[f'{model}_output'][0] == d[f'{model}_output'][-1] == "'":
        d[f'{model}_output'] = d[f'{model}_output'][1:-1]

    ts = d[f'{model}_output'].split('\n')
    ts = [rm_idx(_).strip().split('\n')[0].strip() for _ in ts if match_idx(_)]
    if task != 'SFTTestSeqRec':
        ts = [re.sub(r' *[(,\[](.*)[),\]]$', '', _) for _ in ts]
    ts = [_[1:-1] if _[0] == _[-1] == "'" or _[0] == _[-1] == "\"" else _ for _ in ts if _ != '']
    ts = [_.strip() for _ in ts]
    ts = ts[:k]
    if "SeqRec_Result" in d["input_field_data"]:
        d['input_field_data']['SeqRec_Result'] = d['input_field_data']['SeqRec_Result'][:k]
    d['input_field_data']['item_count'] = k
    d[f'{task}_output_title_list'] = ts
    for idx, _ in enumerate(d[f'{task}_output_title_list']):
        if _ in title2item:
            continue
        for __ in title2item:
            if distance(_, __) <= 3:
                ts[idx] = __
                break
    return d

metric = Metrics([task], k, category2item, title2item)

gpt_seq_rec_data_top10 = []
for _ in res_file:
    old_len = len(gpt_seq_rec_data_top10)
    gpt_seq_rec_data_top10.extend(load_pickle(_))
    print(_, f' | count: {len(gpt_seq_rec_data_top10)-old_len}')
recall, mrr, ndcg, repeat, not_exist, in_history, target_in_history, correct_count = 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0
count = 0

with ProcessPoolExecutor(max_workers=80) as executor:
    results = list(tqdm(executor.map(vague_mapping, gpt_seq_rec_data_top10), total=len(gpt_seq_rec_data_top10)))

for _ in tqdm(results):
    if len(_[f'{task}_output_title_list']) == 0:
        continue
    history = _['input_field_data']['history']
    if ' â\x86\x92 ' in history:
        history = [__[1:-1] for __ in history.split(' â\x86\x92 ')]
    else:
        history = [__[1:-1] for __ in history.split(' → ')]
    assert len(history) > 1
    label = rm_idx(_['output_text'])
    ts = _[f'{task}_output_title_list']
    metric.add_sample(task, _['input_field_data'], ts, [label], vague_mapping=False)
    count += 1
    recall += 1 if label in ts else 0
    mrr += 1/(ts.index(label)+1) if label in ts else 0
    ndcg += 1/math.log2(ts.index(label)+2) if label in ts else 0
    repeat += sum([1 for idx, __ in enumerate(ts) if __ in ts[:idx]])
    not_exist += sum([1 for __ in ts if __ not in title2item])
    in_history += sum([1 for __ in ts if __ in history])
    target_in_history += 1 if label in history else 0
    correct_count += 1 if len(ts) == _['input_field_data']['item_count'] else 0

print(f'''top_k: {k}
count: {count}
recall: {recall/count:.5f}
mrr: {mrr/count:.5f}
ndcg: {ndcg/count:.5f}
repeat: {repeat/count:.5f}
not_exist: {not_exist/count:.5f}
in_history: {in_history/count:.5f}
target_in_history: {target_in_history/count:.5f}
correct_count: {correct_count/count:.5f}''')
metric.print()